In [8]:
from data_ingestion import create_path_to_data
from reading_data import read_map_data, read_localization_data, read_meteo_data
import pandas as pd
import geopandas as gpd


In [15]:
data_path = create_path_to_data()
data = pd.read_csv('c:\\Users\\User\\Desktop\\agh\\analiza w naukach o ziemi git\\data/full_data.csv.gz', compression='gzip', low_memory=False)

In [16]:
data.head()

,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,Suma dobowa opadów [mm],Status pomiaru SMDB,Rodzaj opadu [S/W/ ],Wysokość pokrywy śnieżnej [cm],Status pomiaru PKSN,Wysokość świeżospadłego śniegu [cm],Status pomiaru HSS,Gatunek śniegu [kod],Status pomiaru GATS,Rodzaj pokrywy śnieżnej [kod],Status pomiaru RPSN
0,249180020,WARSZOWICE,2022,12,5,9.5,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN
1,249180020,WARSZOWICE,2022,12,6,6.8,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN
2,249180020,WARSZOWICE,2022,12,7,0.3,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN
3,249180020,WARSZOWICE,2022,12,8,1.0,NaN,S,0,9.0,0,9.0,NaN,9.0,.,NaN
4,249180020,WARSZOWICE,2022,12,9,6.8,NaN,S,0,9.0,0,9.0,NaN,9.0,.,NaN


In [44]:
points = pd.read_csv('c:\\Users\\User\\Desktop\\agh\\analiza w naukach o ziemi git\\data/localization_data.csv.gz', compression='gzip')
points = gpd.GeoDataFrame({"geometry":gpd.points_from_xy(x=points["Długość geograficzna"], y=points["Szerokość geograficzna"]),
                               "ID": points["ID"], "Nazwa": points["Nazwa"], "Rzeka": points["Rzeka"], "Wysokość n.p.m.": points["Wysokość n.p.m."]})
points = points.set_crs(epsg=9702)
points = points.drop(points[points["geometry"].x == points["geometry"].y].index)

In [37]:
mapa = gpd.read_file("c:\\Users\\User\\Desktop\\agh\\analiza w naukach o ziemi git\\data//Wojewodztwa.zip")
mapa = gpd.GeoDataFrame(mapa, geometry="geometry")

In [57]:
region_map = mapa[mapa["JPT_NAZWA_"] == "dolnośląskie"]
points_reg = points[points.within(region_map["geometry"].iloc(0)[0])]

In [58]:
points_reg

,geometry,ID,Nazwa,Rzeka,Wysokość n.p.m.
2,POINT (16.39417 50.33000),250160530,ZIELENIEC,Bystrzyca Dusznicka,840
3,POINT (16.77306 50.21833),250160630,MIĘDZYGÓRZE,Wilczka,800
4,POINT (16.67083 50.15333),250160650,MIĘDZYLESIE,Nysa Kłodzka,453
5,POINT (17.01722 50.75861),250170110,DOBROGOSZCZ,Pluskawa,175
109,POINT (15.82417 50.80361),250150190,KOWARY,Jedlica,430
...,...,...,...,...,...
558,POINT (15.32556 50.90583),250150530,ŚWIERADÓW-ZDRÓJ II,Kwisa,595
560,POINT (16.66833 50.58083),250160910,SREBRNA GÓRA II,Budzówka,380
598,POINT (16.20639 51.41333),251160190,LUBIN,Zimnica,138
600,POINT (14.83556 50.89389),250140020,SIENIAWKA,Nysa Łużycka,234


In [59]:
all_data = points_reg.merge(data, how="left", left_on="ID", right_on="Kod stacji")

In [146]:
all_data[all_data["Kod stacji"].isnull()]

,geometry,ID,Nazwa,Rzeka,Wysokość n.p.m.,Kod stacji,Nazwa stacji,Rok,Miesiąc,Dzień,...,Status pomiaru SMDB,Rodzaj opadu [S/W/ ],Wysokość pokrywy śnieżnej [cm],Status pomiaru PKSN,Wysokość świeżospadłego śniegu [cm],Status pomiaru HSS,Gatunek śniegu [kod],Status pomiaru GATS,Rodzaj pokrywy śnieżnej [kod],Status pomiaru RPSN
245013,POINT (15.78889 50.90028),350150500,JELENIA GÓRA,Bóbr,342,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245014,POINT (16.20778 51.19250),351160415,LEGNICA,Kaczawa,123,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245015,POINT (15.73972 50.73639),350150510,ŚNIEŻKA,Łomniczka,1603,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245016,POINT (16.90000 51.10333),351160424,WROCŁAW-STRACHOWICE,Odra,120,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464864,POINT (16.61417 50.43694),350160520,KŁODZKO,Nysa Kłodzka,356,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594770,POINT (15.69306 50.78917),250150340,BOROWICE,Kacza,660,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594771,POINT (15.68750 50.82556),250150350,PODGÓRZYN,Podgórna,385,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597077,POINT (16.06000 51.67250),251160360,GŁOGÓW,Odra,75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
597078,POINT (15.88889 51.01694),251150410,ŚWIERZAWA,Kaczawa,260,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610374,POINT (16.79278 50.57528),250160360,TARNÓW,Budzówka,296,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
[len(row) for row in all_data.groupby("ID")["Rok"].unique()][60:]

[17, 1, 4, 63, 31, 19, 69, 53, 73, 1, 73, 68, 71, 1, 1, 1, 1, 1, 1]

In [109]:
counted = points.groupby("geometry").count().sort_values("ID")

In [114]:
indexes = counted[counted["ID"]>1].index

In [116]:
for i in indexes:
    print(points[points["geometry"] == i])

                      geometry         ID             Nazwa    Rzeka  \
58   POINT (20.00583 49.24389)  249200540  HALA GĄSIENICOWA  Dunajec   
552  POINT (20.00583 49.24389)  349200628  HALA GĄSIENICOWA  Dunajec   

    Wysokość n.p.m.  
58            1 523  
552           1 523  
                      geometry         ID             Nazwa  Rzeka  \
137  POINT (18.70611 52.36194)  352180345  KOŁO-RADOSZEWICE  Noteć   
519  POINT (18.70611 52.36194)  252180310       RADOSZEWICE  Noteć   

    Wysokość n.p.m.  
137             110  
519             110  
